# **Art_Amp-XL In Context Lora to enhance Lineart Image for Light Novel**

In [ ]:
!pip install diffusers transformers accelerate safetensors huggingface_hub opencv-python

In [ ]:
# Download the required files
!wget https://huggingface.co/kataragi/controlnetXL_line2color/resolve/main/controlnetXL_line2colorV2-fp16.safetensors -O controlnetXL_line2colorV2-fp16.safetensors
!wget https://huggingface.co/kataragi/controlnetXL_line2color/resolve/main/controlnetXL_line2colorV2-lora.safetensors -O controlnetXL_line2colorV2-lora.safetensors

In [ ]:
!wget https://huggingface.co/kataragi/controlnetXL_line2color/resolve/main/model_index.json -O model_index.json

In [ ]:
from diffusers import StableDiffusionXLPipeline, ControlNetModel
from safetensors.torch import load_file
from transformers import CLIPTokenizer, CLIPTextModel
import torch
from PIL import Image

# Load ControlNet weights manually
controlnet_weights_path = "./controlnetXL_line2colorV2-fp16.safetensors"
controlnet_state_dict = load_file(controlnet_weights_path)

# Initialize ControlNet model
controlnet = ControlNetModel.from_config("./model_index.json")
controlnet.load_state_dict(controlnet_state_dict)

# Convert ControlNet to float16 and move to GPU
controlnet = controlnet.to(torch.float16).to("cuda")

# Initialize the SDXL pipeline with ControlNet
pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16
)

# Add ControlNet to the pipeline
pipeline.controlnet = controlnet
pipeline.to("cuda")

# Load tokenizer and text encoder for embedding generation
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to("cuda").eval()


In [ ]:
# Load and preprocess the line art image
lineart_path = "/content/art4.png"  # Path to your uploaded line art
lineart_image = Image.open(lineart_path).convert("RGB")

# Resize the image to match ControlNet's input size
lineart_image = lineart_image.resize((1024, 1024))


In [ ]:
# Define the prompts
#prompt = "Colorize this line art in vibrant anime style with smooth textures and detailed shading."
prompt = "Colorize this anime character lineart with colorful style."
#prompt = "Colorize this line art in vibrant anime style with smooth textures and detailed shading."
negative_prompt = "Blurry, low-quality, unrealistic colors."

# Tokenize the prompts
prompt_tokens = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
negative_prompt_tokens = tokenizer(negative_prompt, return_tensors="pt").input_ids.to("cuda")

# Generate embeddings
text_embeds = text_encoder(prompt_tokens).last_hidden_state
negative_text_embeds = text_encoder(negative_prompt_tokens).last_hidden_state


In [ ]:
# Define generation settings
guidance_scale = 7.5  # Adjust for prompt adherence
num_inference_steps = 100

# Generate the colorized image
result = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=lineart_image,  # Pass the preprocessed image
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale,
    added_cond_kwargs={
        "text_embeds": text_embeds,
        "negative_text_embeds": negative_text_embeds,
    },
).images[0]

# Save and display the result
output_path = "colorized_lineart3.png"
result.save(output_path)
result.show()
